In [1]:
import os
import sys
import time
import logging
import datetime
import glob
import random
import argparse
from typing import List, Dict

os.chdir(r"C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo")

import numpy as np
import torch
from functools import partial
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from sms.exp1.data.formatter import InputFormatter
from sms.src.synthetic_data.note_arr_mod import NoteArrayModifier

In [4]:
dict1 = torch.load(r"C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo\data\exp1\mtc_one_bar_segments_nr.pt")
dict2 = torch.load(r"C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo\data\exp1\maestro_one_bar_segments_nr.pt")
vecs = list(dict1.values()) + list(dict2.values())


C:\Users\cunn2\AppData\Local\Temp\ipykernel_231844\2244089641.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dict1 = torch.load(r"C:\Users\cunn2\OneDrive\DSML\Project\t

In [26]:
v = vecs[0]
np.cumsum(v[:, 0])

array([0.3, 0.8, 1.3, 1.8, 2.3, 2.8, 4. ])

In [23]:
formatter = InputFormatter(make_relative_pitch=False, quantize=True)
modifier = NoteArrayModifier()
aug_config = {
    'use_transposition': False,
    'use_shift_selected_notes_pitch': True,
    'use_change_note_durations': False,
    'use_delete_notes': False,
    'use_insert_notes': False
}

positive_distances = []
negative_distances = []

for i in range(100):
    random_idx = np.random.randint(0, len(vecs))
    anchor = vecs[random_idx]
    anchor_augmented = modifier(anchor, aug_config)
    print(np.sum(anchor[:, 0]))
    print(len(anchor), len(anchor_augmented))
    
    anchor = formatter(anchor)
    anchor_augmented = formatter(anchor_augmented)

    new_idx = random_idx
    while new_idx == random_idx:
        new_idx = np.random.randint(0, len(vecs))

    negative = vecs[new_idx]
    negative = formatter(negative)

    positive_distances.append(np.linalg.norm(anchor - anchor_augmented))
    negative_distances.append(np.linalg.norm(anchor - negative))

print(positive_distances)
print(negative_distances)

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(positive_distances, bins=20, alpha=0.5, label='Positive Distances')
plt.hist(negative_distances, bins=20, alpha=0.5, label='Negative Distances')
plt.xlabel('Distance')
plt.ylabel('Frequency')
plt.title('Histogram of Positive and Negative Distances')
plt.legend()
plt.show()

print(f"Average positive distance: {np.mean(positive_distances):.4f}")
print(f"Average negative distance: {np.mean(negative_distances):.4f}")




[2024-09-24 23:16:33] [INFO ] Shifting note at index 3 by 7 semitones.


4.0
4 4


IndexError: index -1 is out of bounds for axis 0 with size 0